In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import matplotlib.pyplot as plt
import gpytorch
import amort_bnn
import gip_bnn
import gp
import gp_datasets
import tqdm.auto as tqdm

torch.set_default_dtype(torch.float64)

In [ ]:
gpdg = gp_datasets.GPDataGenerator()
num_tasks = 100

In [ ]:
tasks = [gpdg.generate_task(range=[-3.0, 3.0]) for i in range(num_tasks)]
task = tasks[-1]
plt.plot(task["x"], task["y"], label="Ground Truth")
plt.scatter(task["x_context"], task["y_context"], label="Context Points", marker="x")
plt.scatter(task["x_target"], task["y_target"], label="Target Points", marker="+")
plt.xlabel("Input variable")
plt.ylabel("Output variable")
plt.legend()
plt.show()

In [ ]:
amort_bnn_model = amort_bnn.AmortNetwork(
    1,
    [50, 50],
    1,
)

opt = torch.optim.Adam(amort_bnn_model.parameters(), lr=1e-2)

In [ ]:
loss_evo = []
ll_evo = []
kl_evo = []

epoch_iter = tqdm.tqdm(range(1000), "Epoch")
for i, epoch in enumerate(epoch_iter):

    task = tasks[i % num_tasks]

    opt.zero_grad()

    loss, ll, kl, noise = amort_bnn_model.elbo_loss(
        task["x_context"].unsqueeze(1), task["y_context"].unsqueeze(1), num_samples=1
    )
    loss_evo.append(loss.item())
    ll_evo.append(ll.item())
    kl_evo.append(kl.item())

    loss.backward()
    opt.step()

    epoch_iter.set_postfix({"loss": loss, "ll": ll, "kl": kl})

In [ ]:
fig, axs = plt.subplots(3, 1)
fig.set_size_inches(6, 8)
fig.subplots_adjust(hspace=0.5)

axs[0].plot(loss_evo)
axs[0].set_ylabel("ELBO loss")
axs[0].set_xlabel("epoch")
axs[0].set_title("Loss Evolution")

axs[1].plot(ll_evo)
axs[1].set_ylabel("expected log likelihood")
axs[1].set_xlabel("epoch")
axs[1].set_title("Expected Log Likelihood Evolution")

axs[2].plot(kl_evo)
axs[2].set_ylabel("kl")
axs[2].set_xlabel("epoch")
axs[2].set_title("KL Evolution")

plt.show()

In [ ]:
xs = torch.linspace(-2.5, 2.5, 100).unsqueeze(1)
test_task = gpdg.generate_task(range=[-2.0, 2.0])
prediction_samps = (
    amort_bnn_model(
        test_task["x_context"].unsqueeze(1),
        test_task["y_context"].unsqueeze(1),
        x_test=xs,
        num_samples=100,
    )[1]
    .squeeze(-1)
    .T
)

In [ ]:
dataset_size = len(test_task["x_context"])
num_induce = dataset_size  # // 2
rand_perm = torch.randperm(dataset_size)[:num_induce]
inducing_points = test_task["x_context"][rand_perm].unsqueeze(1)

gip_model = gip_bnn.GINetwork(
    1,
    [50, 50],
    1,
    inducing_points,
    nonlinearity=torch.nn.ReLU(),
    last_layer_pseud_obs=test_task["y_context"][rand_perm]
    .unsqueeze(-1)
    .transpose(-1, -2),
)

opt = torch.optim.Adam(gip_model.parameters(), lr=1e-2)

loss_evo = []
ll_evo = []
kl_evo = []

epoch_iter = tqdm.tqdm(range(500), "Epoch")
for epoch in epoch_iter:  # epochs
    opt.zero_grad()

    loss, ll, kl, noise = gip_model.elbo_loss(
        test_task["x_context"].unsqueeze(1),
        test_task["y_context"].unsqueeze(1),
        num_samples=1,
    )
    loss_evo.append(loss.item())
    ll_evo.append(ll.item())
    kl_evo.append(kl.item())

    loss.backward()
    opt.step()

    epoch_iter.set_postfix({"loss": loss, "ll": ll, "kl": kl})


fig, axs = plt.subplots(3, 1)
fig.set_size_inches(6, 8)
fig.subplots_adjust(hspace=0.5)

axs[0].plot(loss_evo)
axs[0].set_ylabel("ELBO loss")
axs[0].set_xlabel("epoch")
axs[0].set_title("Loss Evolution")

axs[1].plot(ll_evo)
axs[1].set_ylabel("expected log likelihood")
axs[1].set_xlabel("epoch")
axs[1].set_title("Expected Log Likelihood Evolution")

axs[2].plot(kl_evo)
axs[2].set_ylabel("kl")
axs[2].set_xlabel("epoch")
axs[2].set_title("KL Evolution")

plt.show()

In [ ]:
gip_prediction_samps = gip_model(xs, num_samples=100)[0].squeeze(-1).T

In [ ]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
gp_model = gp.GPModel(test_task["x_context"], test_task["y_context"], likelihood)

gp_model.train()
likelihood.train()
opt = torch.optim.Adam(gp_model.parameters(), lr=1e-1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, gp_model)
mll_evo = []

epoch_iter = tqdm.tqdm(range(150), "Epoch")
for epoch in epoch_iter:
    opt.zero_grad()
    output = gp_model(test_task["x_context"])
    loss = -mll(output, test_task["y_context"])
    mll_evo.append(loss.item())
    loss.backward()
    opt.step()
    epoch_iter.set_postfix({"loss": loss})

plt.plot(mll_evo)
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Marginal Log Likelihood")
plt.show()

In [ ]:
gp_model.eval()
gp_prediction_samps = (
    gp_model(xs)
    .sample(
        torch.Size(
            [
                100,
            ]
        )
    )
    .T
)

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(18, 5)

axs[0].plot(
    xs,
    prediction_samps.detach().numpy()[:, :-1],
    color="blue",
    linewidth=0.5,
    alpha=0.15,
)
axs[0].plot(
    xs,
    prediction_samps.detach().numpy()[:, -1],
    color="blue",
    linewidth=0.5,
    alpha=0.15,
    label="Predictive Sample",
)
axs[0].set_title("Amortised Model Prediction Samples")
axs[0].set_xlabel("Input Variable")
axs[0].set_ylabel("Output Variable")
axs[0].scatter(
    test_task["x_context"],
    test_task["y_context"],
    marker="x",
    label="Training Data",
    color="red",
    linewidth=0.5,
)

axs[0].legend()
axs[0].set_ylim(-4.0, 4.0)
axs[0].set_xlim(-2.5, 2.5)

axs[1].plot(
    xs,
    gip_prediction_samps.detach().numpy()[:, :-1],
    color="blue",
    linewidth=0.5,
    alpha=0.15,
)
axs[1].plot(
    xs,
    gip_prediction_samps.detach().numpy()[:, -1],
    color="blue",
    linewidth=0.5,
    alpha=0.15,
    label="Predictive Sample",
)
axs[1].set_title("Global Inducing Point Model Prediction Samples")
axs[1].set_xlabel("Input Variable")
axs[1].set_ylabel("Output Variable")
axs[1].scatter(
    test_task["x_context"],
    test_task["y_context"],
    marker="x",
    label="Training Data",
    color="red",
    linewidth=0.5,
)

axs[1].legend()
axs[1].set_ylim(-4.0, 4.0)
axs[1].set_xlim(-2.5, 2.5)

axs[2].plot(
    xs,
    gp_prediction_samps.detach().numpy()[:, :-1],
    color="blue",
    linewidth=0.5,
    alpha=0.15,
)
axs[2].plot(
    xs,
    gp_prediction_samps.detach().numpy()[:, -1],
    color="blue",
    linewidth=0.5,
    alpha=0.15,
    label="Predictive Sample",
)
# axs[2].plot(xs, gp_prediction_samps.detach().numpy())
axs[2].set_title("Gaussian Process Model Prediction Samples")
axs[2].set_xlabel("Input Variable")
axs[2].set_ylabel("Output Variable")
axs[2].scatter(
    test_task["x_context"],
    test_task["y_context"],
    marker="x",
    label="Training Data",
    color="red",
    linewidth=0.5,
)

axs[2].legend()
axs[2].set_ylim(-4.0, 4.0)
axs[2].set_xlim(-2.5, 2.5)

plt.show()